In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from typing import List
from bs4 import BeautifulSoup
import requests

In [30]:
import pandas as pd

In [45]:
def scrape_largest_companies(num_pages: int= 58) -> pd.DataFrame:
    """Scrapes name, ticker symbols and country of top companies based on market cap.
    
    Parameters
    ----------
    num_pages: int
        number of pages to scrape, each page has 100 symbols, max is 
    
    Returns
    -------
        A pandas DataFrame
    """
    list_tickers = []
    list_names = []
    list_countries = []
    if num_pages == 1:
        url = 'https://companiesmarketcap.com/'
        html = requests.get(url).text
        soup = BeautifulSoup(html, 'html.parser')
        list_tickers = [e.text for e in soup.select('div.company-code')]
    else: 
        for num_page in range(num_pages):
            url = f"https://companiesmarketcap.com/page/{num_page+1}/"
            html = requests.get(url).text
            soup = BeautifulSoup(html, 'html.parser')
            list_ticker_page = [e.text for e in soup.select('div.company-code')]
            list_names_page = [e.text for e in soup.select('div.company-name')]
            list_countries_page = [e.text for e in soup.select('span.responsive-hidden')]
            list_countries_page = list_countries_page[1:]
            list_tickers = list_tickers + list_ticker_page
            list_names = list_names + list_names_page 
            list_countries = list_countries + list_countries_page
            companies_dict = {
        "ticker": list_tickers,
        "name": list_names,
        "country": list_countries,
    }
            
    return pd.DataFrame(companies_dict, columns=companies_dict.keys())

In [47]:
assets_df = scrape_largest_companies()

In [52]:
assets_df

,ticker,name,country,is_etf
0,AAPL,Apple,USA,False
1,MSFT,Microsoft,USA,False
2,2222.SR,Saudi Aramco,S. Arabia,False
3,GOOG,Alphabet (Google),USA,False
4,AMZN,Amazon,USA,False
...,...,...,...,...
5744,JW,Justworks,USA,False
5745,RHDM,Rhodium Enterprises,USA,False
5746,SMSA,Samsara Vision,USA,False
5747,VDNT,Verdant Earth Technologies,Australia,False


In [51]:
assets_df["is_etf"]=[False for _ in range(len(assets_df.index))]

In [3]:
import config
import psycopg2

In [7]:
connection = psycopg2.connect(
            host=config.DB_HOST,
            database=config.DB_NAME,
            user=config.DB_USER,
            password=config.DB_PASSWORD,
        )
cur = connection.cursor()

In [35]:
sql_query= """SELECT id, ticker FROM assets"""
cur.execute(query = sql_query)

In [36]:
row = cur.fetchall()
row

[(2, 'AAPL'),
 (3, 'MSFT'),
 (4, '2222.SR'),
 (5, 'GOOG'),
 (6, 'AMZN'),
 (7, 'TSLA'),
 (8, 'FB'),
 (9, 'BRK-A'),
 (10, 'TSM'),
 (11, 'NVDA'),
 (12, 'TCEHY'),
 (13, 'V'),
 (14, 'UNH'),
 (15, 'JNJ'),
 (16, 'JPM'),
 (17, '005930.KS'),
 (18, 'LVMUY'),
 (19, 'PG'),
 (20, '600519.SS'),
 (21, 'WMT'),
 (22, 'BAC'),
 (23, 'NSRGY'),
 (24, 'HD'),
 (25, 'MA'),
 (26, 'BABA'),
 (27, 'RHHBY'),
 (28, 'XOM'),
 (29, 'PFE'),
 (30, 'ASML'),
 (31, 'TM'),
 (32, 'KO'),
 (33, '1398.HK'),
 (34, 'DIS'),
 (35, 'CVX'),
 (36, 'PEP'),
 (37, 'CSCO'),
 (38, 'ADBE'),
 (39, 'ABBV'),
 (40, 'LLY'),
 (41, 'TMO'),
 (42, 'OR.PA'),
 (43, 'CMCSA'),
 (44, 'NKE'),
 (45, 'RELIANCE.NS'),
 (46, 'VZ'),
 (47, 'ABT'),
 (48, 'NVO'),
 (49, 'AVGO'),
 (50, 'ORCL'),
 (51, 'CRM'),
 (52, 'COST'),
 (53, 'ACN'),
 (54, 'INTC'),
 (55, '3968.HK'),
 (56, '300750.SZ'),
 (57, 'WFC'),
 (58, 'MRK'),
 (59, 'DHR'),
 (60, 'NVS'),
 (61, 'PYPL'),
 (62, 'CICHY'),
 (63, 'TCS.NS'),
 (64, 'MCD'),
 (65, 'RYDAF'),
 (66, 'T'),
 (67, 'QCOM'),
 (68, 'AZN'),
 (69,

In [38]:
for id, ticker in row:
    print(id)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
27

In [16]:
import pandas as pd

In [25]:
for tick in list_tickers[:10]:
    ticker = yf.Ticker(tick)
    prices= ticker.history(period="max", auto_adjust=False)
    prices["ticker"] = tick
    prices.index = pd.to_datetime(prices.index, format='%Y-%m-%d')
    prices.index= prices.index.rename("time")
    prices = prices.rename(columns={
                            'Open': 'open',
                            'Close': 'close',
                            'High': 'high',
                            'Low': 'low',
                            'Volume': 'volume'
                            })
    prices = prices.drop(["Adj Close", "Dividends", "Stock Splits"], axis =1)
prices

,open,high,low,close,volume,ticker
time,,,,,,
1999-01-22,0.437500,0.488281,0.388021,0.410156,271468800.0,NVDA
1999-01-25,0.442708,0.458333,0.410156,0.453125,51048000.0,NVDA
1999-01-26,0.458333,0.467448,0.411458,0.417969,34320000.0,NVDA
1999-01-27,0.419271,0.429688,0.395833,0.416667,24436800.0,NVDA
1999-01-28,0.416667,0.419271,0.412760,0.415365,22752000.0,NVDA
...,...,...,...,...,...,...
2022-01-14,263.000000,271.970001,262.100006,269.420013,39522800.0,NVDA
2022-01-18,262.600006,266.380005,257.700012,259.029999,42427000.0,NVDA
2022-01-19,260.790009,265.429993,250.520004,250.669998,48831500.0,NVDA


In [32]:
[row for row in prices.itertuples(index=False, name=None)]

[(0.4375,
  0.4882810115814209,
  0.38802099227905273,
  0.4101560115814209,
  271468800.0,
  'NVDA'),
 (0.44270798563957214,
  0.45833298563957214,
  0.4101560115814209,
  0.453125,
  51048000.0,
  'NVDA'),
 (0.45833298563957214,
  0.46744799613952637,
  0.41145798563957214,
  0.4179689884185791,
  34320000.0,
  'NVDA'),
 (0.41927099227905273,
  0.4296880066394806,
  0.39583298563957214,
  0.41666701436042786,
  24436800.0,
  'NVDA'),
 (0.41666701436042786,
  0.41927099227905273,
  0.4127599895000458,
  0.4153650104999542,
  22752000.0,
  'NVDA'),
 (0.4153650104999542,
  0.41666701436042786,
  0.39583298563957214,
  0.39583298563957214,
  24403200.0,
  'NVDA'),
 (0.39583298563957214,
  0.40625,
  0.39583298563957214,
  0.40364599227905273,
  15470400.0,
  'NVDA'),
 (0.39583298563957214,
  0.40625,
  0.36067700386047363,
  0.37239599227905273,
  26409600.0,
  'NVDA'),
 (0.3671880066394806,
  0.38541701436042786,
  0.36458298563957214,
  0.38020798563957214,
  7512000.0,
  'NVDA'),
 (0.

In [5]:
cur.close()
connection.close()

In [2]:
import yfinance as yf
msft = yf.Ticker("MSFT")
msft.info.keys()

dict_keys(['zip', 'sector', 'fullTimeEmployees', 'longBusinessSummary', 'city', 'phone', 'state', 'country', 'companyOfficers', 'website', 'maxAge', 'address1', 'industry', 'ebitdaMargins', 'profitMargins', 'grossMargins', 'operatingCashflow', 'revenueGrowth', 'operatingMargins', 'ebitda', 'targetLowPrice', 'recommendationKey', 'grossProfits', 'freeCashflow', 'targetMedianPrice', 'currentPrice', 'earningsGrowth', 'currentRatio', 'returnOnAssets', 'numberOfAnalystOpinions', 'targetMeanPrice', 'debtToEquity', 'returnOnEquity', 'targetHighPrice', 'totalCash', 'totalDebt', 'totalRevenue', 'totalCashPerShare', 'financialCurrency', 'revenuePerShare', 'quickRatio', 'recommendationMean', 'exchange', 'shortName', 'longName', 'exchangeTimezoneName', 'exchangeTimezoneShortName', 'isEsgPopulated', 'gmtOffSetMilliseconds', 'quoteType', 'symbol', 'messageBoardId', 'market', 'annualHoldingsTurnover', 'enterpriseToRevenue', 'beta3Year', 'enterpriseToEbitda', '52WeekChange', 'morningStarRiskRating', 'f

In [ ]:
cvs = yf.Ticker("CVS")
cvs_data = cvs.history(period="max")

In [3]:
import psycopg2
import config
import yfinance as yf
import pandas as pd


In [34]:
create_prices_table_sql = """CREATE TABLE IF NOT EXISTS prices (
    assets_id INTEGER NOT NULL,
    time TIMESTAMP WITHOUT TIME ZONE NOT NULL,
    ticker text NULL,
    open double precision NULL,
    low double precision NULL,
    high double precision NULL,
    close double precision NULL,
    volume int NULL,
    market_cap double precision NULL,
    PRIMARY KEY (assets_id, time),
    CONSTRAINT fk_assets FOREIGN KEY (assets_id) REFERENCES assets(id)
);"""
sql_query = """SELECT id, ticker FROM assets"""
try:
    connection = psycopg2.connect(
        host=config.DB_HOST,
        database=config.DB_NAME,
        user=config.DB_USER,
        password=config.DB_PASSWORD,
    )
    cur = connection.cursor()
    cur.execute(create_prices_table_sql)
    cur.execute(query=sql_query)
    rows = cur.fetchall()
    for id, ticker in rows[:1]:
        print(id,ticker)
        yfticker = yf.Ticker(ticker)
        prices = yfticker.history(period="max", auto_adjust=False)
        prices["assets_id"] = id
        prices["ticker"] = ticker
        print(prices)
        prices.index = prices.index.date
        prices.index = prices.index.rename("time")
        prices = prices.rename(
            columns={
                "Open": "open",
                "Close": "close",
                "High": "high",
                "Low": "low",
                "Volume": "volume",
            }
        )
        prices = prices.drop(["Adj Close", "Dividends", "Stock Splits"], axis=1)
        try:
            for row in prices.itertuples(index=True, name=None):
                print(row)
                cur.execute(
                    """INSERT INTO prices (time, open, high, low, close, volume, assets_id, ticker)
                VALUES (%s,%s,%s,%s,%s,%s,%s,%s)""",
                    row,
                )
        except psycopg2.errors.UniqueViolation:
                pass
        except Exception as e:
            print(str(e.__class__) + "when inserting")
            
    # mgr = CopyManager(conn=connection, table="prices", cols=prices.columns)
    # mgr.copy([row for row in prices.itertuples(index=True, name=None)])
    connection.commit()
except (Exception, psycopg2.DatabaseError) as error:
    print(error)
finally:
    cur.close()
    connection.close()

2 AAPL
                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1980-12-12    0.128348    0.128906    0.128348    0.128348    0.100453   
1980-12-15    0.122210    0.122210    0.121652    0.121652    0.095213   
1980-12-16    0.113281    0.113281    0.112723    0.112723    0.088224   
1980-12-17    0.115513    0.116071    0.115513    0.115513    0.090408   
1980-12-18    0.118862    0.119420    0.118862    0.118862    0.093029   
...                ...         ...         ...         ...         ...   
2022-01-14  171.339996  173.779999  171.089996  173.070007  173.070007   
2022-01-18  171.509995  172.539993  169.410004  169.800003  169.800003   
2022-01-19  170.000000  171.080002  165.940002  166.229996  166.229996   
2022-01-20  166.979996  169.679993  164.179993  164.509995  164.509995   
2022-01-21  164.419998  166.330002  162.300003  162.410004  162.410004   

               Volume  Dividen

In [15]:
prices_list = []
sql_query = """SELECT id, ticker FROM assets"""
connection = psycopg2.connect(
            host=config.DB_HOST,
            database=config.DB_NAME,
            user=config.DB_USER,
            password=config.DB_PASSWORD,
        )
cur = connection.cursor()
cur.execute(query=sql_query)
rows = cur.fetchall()
for id, ticker in rows[:50]:
    yfticker = yf.Ticker(ticker)
    prices = yfticker.history(period="max", auto_adjust=False)
    prices["assets_id"] = id
    prices["ticker"] = ticker
    # prices.index = pd.to_datetime(prices.index, format="%Y-%m-%d")
    prices.index = prices.index.date
    prices.index = prices.index.rename("time")
    prices = prices.rename(
        columns={
            "Open": "open",
            "Close": "close",
            "High": "high",
            "Low": "low",
            "Volume": "volume",
        }
    )
    prices = prices.drop(["Adj Close", "Dividends", "Stock Splits"], axis=1)
    prices_list = [row for row in prices.itertuples(index=True, name=None)]

In [20]:
for row in prices.itertuples(index=True, name=None):
    print(row)

(datetime.date(2004, 6, 23), 3.75, 4.324999809265137, 3.6875, 4.300000190734863, 43574400, 51, 'CRM')
(datetime.date(2004, 6, 24), 4.387499809265137, 4.422500133514404, 4.125, 4.190000057220459, 8887200, 51, 'CRM')
(datetime.date(2004, 6, 25), 4.127500057220459, 4.1875, 3.947499990463257, 3.950000047683716, 6710000, 51, 'CRM')
(datetime.date(2004, 6, 28), 4.0, 4.052499771118164, 3.859999895095825, 4.0, 2270800, 51, 'CRM')
(datetime.date(2004, 6, 29), 4.0, 4.175000190734863, 3.9574999809265137, 4.099999904632568, 2112000, 51, 'CRM')
(datetime.date(2004, 6, 30), 4.099999904632568, 4.235000133514404, 4.007500171661377, 4.017499923706055, 2087600, 51, 'CRM')
(datetime.date(2004, 7, 1), 4.019999980926514, 4.050000190734863, 3.9749999046325684, 4.007500171661377, 1754800, 51, 'CRM')
(datetime.date(2004, 7, 2), 4.025000095367432, 4.244999885559082, 4.012499809265137, 4.244999885559082, 993200, 51, 'CRM')
(datetime.date(2004, 7, 6), 4.232500076293945, 4.25, 4.119999885559082, 4.25, 1216800, 51

In [17]:
prices

,open,high,low,close,volume,assets_id,ticker
time,,,,,,,
2004-06-23,3.750000,4.325000,3.687500,4.300000,43574400,51,CRM
2004-06-24,4.387500,4.422500,4.125000,4.190000,8887200,51,CRM
2004-06-25,4.127500,4.187500,3.947500,3.950000,6710000,51,CRM
2004-06-28,4.000000,4.052500,3.860000,4.000000,2270800,51,CRM
2004-06-29,4.000000,4.175000,3.957500,4.100000,2112000,51,CRM
...,...,...,...,...,...,...,...
2022-01-14,228.029999,233.289993,227.610001,231.229996,5804100,51,CRM
2022-01-18,228.449997,230.729996,225.300003,226.059998,6032000,51,CRM
2022-01-19,227.470001,232.210007,226.169998,226.270004,5332600,51,CRM


In [16]:
prices_list[-1]

(datetime.date(2022, 1, 21),
 223.05999755859375,
 224.77000427246094,
 218.25,
 218.6300048828125,
 9116900,
 51,
 'CRM')